<a href="https://colab.research.google.com/github/Fuenfgeld/2022TeamADataEngineeringBC/blob/PySpark/PySparkTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

In [ ]:
!wget -cq https://github.com/domoritz/maps/blob/master/data/iris.json

In [ ]:
!wget -cq https://github.com/Fuenfgeld/2022TeamADataEngineeringBC/blob/PySpark/iris2.json

##1. Loading Data

Before we can analyze data we have to load it into our working environment. PySpark has a lot of functions that can deal with all kinds of formats from `.csv` to `.json`. The basic unit of data storage in PySpark is the so called `DataFrame` class.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
df1 = spark.read.option("multiline",True).json('iris-dataset-json-version.json')
print(f"Object Type: {type(df1)}\n")
print("Column Info:")
df.printSchema()
print("Overview Dataframe:")
df1.show(10)

In [ ]:
df.select("petalLength").describe().show()

##2. Basic transformations
Some of the most basic functionalities of tables are that we can access specific chunks of the tables rows and columns as well as new rows and columns.

`DataFrame.collect()` collects the distributed data to the driver side as the local data in Python. Note that this can throw an out-of-memory error when the dataset is too large to fit in the driver side because it collects all the data from executors to the driver side.

In [ ]:
local_df1 = df1.collect()
local_df1[0]

If we want to access specific columns we can use `.NameOfColumn`

In [ ]:
petalLength = df1.petalLength
petalWidth = df1.petalWidth
df1.select(petalLength, petalWidth).show(5)

Suppose we have similar datasets from multiple sources. Wouldn't it be practical to combine them into one table ? `pyspark` provides such a funcionality via the `.union()` method which is equivalent to `UNION ALL` in SQL.

In [ ]:
df2 = spark.read.json('iris2.json')
df2.show()
df1.union(df2)

In case we want to add columns we can do so via the `.withColumn()` method. Note that we have to specify the name of the column which is in this case `petalSum`. Usually the new column is a function of one or more of the old columns. 

In [ ]:
df_extraCol = df1.withColumn('petalSum', df1.petalWidth + df1.petalLength)
df_extraCol.show(5)

In order to perform the opposite operation, i.e removing a column `.drop()` can be used. In contrast to `.select()`, this method removes the specified column completely instead of returning it as slice ot the table.

In [ ]:
df1 = df_extraCol.drop(df_extraCol.petalSum)
df1.show(5)

You learned how to perform some basic transformations of the table, but what if you want to look up values not based on indices but rather on criteria such as a certain column's entry being bigger than some threshold? In the next chapter we are going to take a look at how to select rows via user given conditions.